## The second step is completing the 4-step IBC connection handshake between terra and osmosis - 1) init (on terra), 2) try (on osmosis), 3) ack (on terra), 4) confirm (on osmosis).

## With an IBC connection in-place, we lay the foundation for terra/osmosis smart contract pairs to communicate.

In [24]:
#imports

import pandas as pd
import json
import os
import sys
import base64
import requests
import subprocess
import math
import hashlib
import bech32
import time

from dateutil.parser import parse
from datetime import datetime, timedelta
from ecdsa import SECP256k1, SigningKey
from ecdsa.util import sigencode_string_canonize
from bech32 import bech32_decode, bech32_encode, convertbits
from google.protobuf.timestamp_pb2 import Timestamp as googTimestamp

from terra_sdk.client.lcd import LCDClient
from terra_sdk.core.wasm import MsgStoreCode, MsgInstantiateContract, MsgExecuteContract
from terra_sdk.core.bank import MsgSend
from terra_sdk.core.fee import Fee
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.core.bech32 import get_bech
from terra_sdk.core import AccAddress, Coin, Coins
from terra_sdk.client.lcd.api.tx import CreateTxOptions, SignerOptions
from terra_sdk.client.localterra import LocalTerra
from terra_sdk.core.wasm.data import AccessConfig
from terra_sdk.client.lcd.api._base import BaseAsyncAPI, sync_bind

from terra_proto.cosmwasm.wasm.v1 import AccessType
from terra_proto.cosmos.tx.v1beta1 import Tx, TxBody, AuthInfo, SignDoc, SignerInfo, ModeInfo, ModeInfoSingle, BroadcastTxResponse
from terra_proto.cosmos.base.abci.v1beta1 import TxResponse
from terra_proto.cosmos.tx.signing.v1beta1 import SignMode
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse
from terra_proto.ibc.core.channel.v1 import MsgChannelOpenInit, Channel, State, Order, Counterparty, MsgChannelOpenTry, MsgChannelOpenAck, MsgChannelOpenConfirm, QueryUnreceivedPacketsRequest, QueryUnreceivedPacketsResponse, QueryPacketCommitmentRequest, QueryPacketCommitmentResponse, Packet, QueryNextSequenceReceiveRequest, QueryNextSequenceReceiveResponse, MsgRecvPacket, MsgTimeout, QueryUnreceivedAcksRequest, QueryUnreceivedAcksResponse, MsgAcknowledgement
from terra_proto.ibc.core.connection.v1 import MsgConnectionOpenInit, Counterparty as ConnectionCounterParty, Version, MsgConnectionOpenTry, MsgConnectionOpenAck, MsgConnectionOpenConfirm
from terra_proto.ibc.lightclients.tendermint.v1 import ClientState, ConsensusState, Fraction, Header
from terra_proto.ics23 import HashOp, LengthOp, LeafOp, InnerOp, ProofSpec, InnerSpec, CommitmentProof, ExistenceProof, NonExistenceProof, BatchProof, CompressedBatchProof, BatchEntry, CompressedBatchEntry, CompressedExistenceProof, CompressedNonExistenceProof
from terra_proto.ibc.core.commitment.v1 import MerkleRoot, MerklePrefix, MerkleProof
from terra_proto.tendermint.types import ValidatorSet, Validator, SignedHeader, Header as tendermintHeader, Commit, BlockId, PartSetHeader, CommitSig, BlockIdFlag
from terra_proto.tendermint.version import Consensus
from terra_proto.tendermint.crypto import PublicKey
from betterproto.lib.google.protobuf import Any
from betterproto import Timestamp

#misc helper functions
sys.path.append(os.path.join(os.path.dirname(__name__), '..', 'scripts'))

from helpers import proto_to_binary, timestamp_string_to_proto, stargate_msg, create_ibc_client, fetch_chain_objects, bech32_to_hexstring, hexstring_to_bytes, bech32_to_b64

In [3]:
#setup lcd clients, rpc urls, wallets
(terra, wallet, terra_rpc_url, terra_rpc_header) = fetch_chain_objects("pisco-1")
(osmo, osmo_wallet, osmo_rpc_url, osmo_rpc_header) = fetch_chain_objects("osmo-test-4")

https://rpc.pisco.terra.setten.io/a0a3abea69544a99a67700ce2c7926fb
osmo16kknw79vd8rws905phrq6t5ex8vzxdhvc0gsmp


## These are the protobuf classes specific for setting up the IBC connection. One noteworthy protobuf class that will see continual usage now, and in later stages, is MsgUpdateClient, which basically refreshes the light client information (ie the counterpart chain tendermint state).

In [4]:
#terra_proto imports
from terra_proto.ibc.core.connection.v1 import MsgConnectionOpenInit, Counterparty as ConnectionCounterParty, Version, MsgConnectionOpenTry, MsgConnectionOpenAck, MsgConnectionOpenConfirm
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse

## Load client_id's from previous step; if we don't find it, we'll use default client_id's built by the authors.

In [5]:
#load context.json
context = {}
# Open the file for reading
with open("context.json", "r") as f:
    # Load the dictionary from the file
    context = json.load(f)
    
client_id_on_terra = context["client_id_on_terra"]
client_id_on_osmo = context["client_id_on_osmo"]

## IBC connection step 1 - MsgConnectionOpenInit (called on terra)

In [7]:
#fabricate message & dispatch via stargate
msg = MsgConnectionOpenInit(
  client_id=client_id_on_terra,
  counterparty=ConnectionCounterParty(
    client_id=client_id_on_osmo,
    prefix=MerklePrefix(
      key_prefix=bytes("ibc", "ascii"),
    )
  ),
  version=Version(
    identifier="1",
    features=["ORDER_ORDERED", "ORDER_UNORDERED"],
  ),
  delay_period=0,
  signer=wallet.key.acc_address
)

result = stargate_msg("/ibc.core.connection.v1.MsgConnectionOpenInit", msg, wallet, terra)
result_df = pd.DataFrame(result["tx_response"]["logs"][0]["events"][0]["attributes"])
connection_id_on_terra = result_df[result_df["key"]=="connection_id"]["value"].values[0]

print(f"connection_id on terra: {connection_id_on_terra}")

connection_id on terra: connection-214


## IBC connection step 2 - MsgConnectionOpenTry (called on osmosis)

In [30]:
#update the client_id on osmosis (ie, refresh osmosis's knowledge of terra's tendermint state)
#only this cell will show the full gory details; the rest will just call the helper function

#gather tendermint/consensus/validator information on terra's current state
tendermint_info_on_other_chain = terra.tendermint.block_info()

timestamp = googTimestamp()
timestamp.FromJsonString(tendermint_info_on_other_chain["block"]["header"]["time"])
print(f"timestamp: {timestamp} \n\n")

print("source current tendermint:")
print(tendermint_info_on_other_chain)
print("\n\n")

validator_info_on_other_chain = terra.tendermint.validator_set(height=int(tendermint_info_on_other_chain["block"]["header"]["height"]))
commit_info_on_other_chain = requests.get(f"{terra_rpc_url}/commit", headers=terra_rpc_header, params={"height": tendermint_info_on_other_chain["block"]["header"]["height"]}).json()

print("\n\ncommit_info:\n")
print(commit_info_on_other_chain)
print("\n\n")

resp = requests.get(f"{terra_rpc_url}/blockchain", headers=terra_rpc_header, params={"minHeight": tendermint_info_on_other_chain["block"]["header"]["height"], "maxHeight": tendermint_info_on_other_chain["block"]["header"]["height"]}).json()
block_proposer_on_other_chain = resp["result"]["block_metas"][0]["header"]["proposer_address"]

proposer_info = None
for x in validator_info_on_other_chain["validators"]:
    if block_proposer_on_other_chain == bech32_to_hexstring(x["address"]).upper():
        proposer_info = x

#gather osmosis's understanding of terra's tendermint/consensus/validator information 
client_state_of_other_chain_on_my_chain = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")
validator_info_of_other_chain_on_my_chain = terra.tendermint.validator_set(height=int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"])+1)
block_proposer_of_other_chain_on_my_chain = requests.get(f"{terra_rpc_url}/blockchain", headers=terra_rpc_header, params={"minHeight": int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"])+1, "maxHeight": int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"])+1}).json()["result"]["block_metas"][0]["header"]["proposer_address"]

trusted_proposer_info = None
for x in validator_info_of_other_chain_on_my_chain["validators"]:
    if block_proposer_of_other_chain_on_my_chain == bech32_to_hexstring(x["address"]).upper():
        trusted_proposer_info = x

version_app = 0 if "app" not in commit_info_on_other_chain["result"]["signed_header"]["header"]["version"].keys() else int(commit_info_on_other_chain["result"]["signed_header"]["header"]["version"]["app"])

#fabricate the MsgUpdateClient message; this updates osmosis's knowledge of terra's tendermint/consensus state
msg = MsgUpdateClient(
    signer=osmo_wallet.key.acc_address,
    client_id=client_id_on_osmo,
    header=Any(
      type_url="/ibc.lightclients.tendermint.v1.Header",
      value=Header(
        signed_header=SignedHeader(
          header=tendermintHeader(
            version=Consensus(
              block=int(commit_info_on_other_chain["result"]["signed_header"]["header"]["version"]["block"]),
              app=version_app,
            ),
            chain_id=terra.chain_id,
            height=int(commit_info_on_other_chain["result"]["signed_header"]["header"]["height"]),
            time=Timestamp(timestamp.seconds, timestamp.nanos),
            last_block_id=BlockId(
              hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_block_id"]["hash"]),
              part_set_header=PartSetHeader(
                total=int(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_block_id"]["parts"]["total"]),
                hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_block_id"]["parts"]["hash"]),
              ),
            ),
            last_commit_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_commit_hash"]),
            data_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["data_hash"]),
            validators_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["validators_hash"]),
            next_validators_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["next_validators_hash"]),
            consensus_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["consensus_hash"]),
            app_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["app_hash"]),
            last_results_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_results_hash"]),
            evidence_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["evidence_hash"]),
            proposer_address=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["proposer_address"]),
          ),
          commit=Commit(
            height=int(commit_info_on_other_chain["result"]["signed_header"]["commit"]["height"]),
            round=int(commit_info_on_other_chain["result"]["signed_header"]["commit"]["round"]),
            block_id=BlockId(
              hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["commit"]["block_id"]["hash"]),
              part_set_header=PartSetHeader(
                total=int(commit_info_on_other_chain["result"]["signed_header"]["commit"]["block_id"]["parts"]["total"]),
                hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["commit"]["block_id"]["parts"]["hash"]),
              )
            ),
            signatures=[
              CommitSig(
                block_id_flag=BlockIdFlag(x["block_id_flag"]),
                validator_address=hexstring_to_bytes(x["validator_address"]),
                timestamp=timestamp_string_to_proto(x["timestamp"]),
                signature=base64.b64decode(x["signature"]) if x["signature"] is not None else None,
              )
              for x in commit_info_on_other_chain["result"]["signed_header"]["commit"]["signatures"]
            ],
          ),
        ),
        validator_set=ValidatorSet(
          validators=[
            Validator(
              address=base64.b64decode(bech32_to_b64(x["address"])),
              pub_key=PublicKey(
                ed25519=base64.b64decode(x["pub_key"]["key"])
              ) if "ed25519" in x["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(x["pub_key"]["key"])
              ),
              voting_power=int(x["voting_power"]),
            ) for x in validator_info_on_other_chain["validators"]
          ],
          proposer=Validator(
            address=base64.b64decode(bech32_to_b64(proposer_info["address"])),
            pub_key=PublicKey(
              ed25519=base64.b64decode(proposer_info["pub_key"]["key"])
            ) if "ed25519" in proposer_info["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(proposer_info["pub_key"]["key"])
            ),
            voting_power=int(proposer_info["voting_power"]),
          ) 
          ,
          total_voting_power=sum([int(x["voting_power"]) for x in validator_info_on_other_chain["validators"]]),
        ),
        trusted_height=Height(
          revision_number=int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_number"]),
          revision_height=int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"]),
        ),
        trusted_validators=ValidatorSet(
          validators=[
            Validator(
              address=base64.b64decode(bech32_to_b64(x["address"])),
              pub_key=PublicKey(
                ed25519=base64.b64decode(x["pub_key"]["key"])
              ) if "ed25519" in x["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(x["pub_key"]["key"])
              ),
              voting_power=int(x["voting_power"]),
            ) for x in validator_info_of_other_chain_on_my_chain["validators"]
          ],
          proposer=Validator(
            address=base64.b64decode(bech32_to_b64(trusted_proposer_info["address"])),
            pub_key=PublicKey(
              ed25519=base64.b64decode(trusted_proposer_info["pub_key"]["key"])
            ) if "ed25519" in trusted_proposer_info["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(trusted_proposer_info["pub_key"]["key"])
            ),
            voting_power=int(trusted_proposer_info["voting_power"]),
          ) 
          ,
          total_voting_power=sum([int(x["voting_power"]) for x in validator_info_of_other_chain_on_my_chain["validators"]]),
        ),
      ).SerializeToString(),
    ),
)

print(f"""

update client msg:

{ msg.to_dict()}

""")

update_client_on_osmo_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", msg, osmo_wallet, osmo)
header_height = Header.FromString(msg.header.value).signed_header.header.height

timestamp: seconds: 1671065446
nanos: 776126771
 


source current tendermint:
{'block': {'header': {'version': {'block': '11', 'app': '0'}, 'chain_id': 'pisco-1', 'height': '3200591', 'time': '2022-12-15T00:50:46.776126771Z', 'last_block_id': {'hash': 'ENA6auAlxtXhj6EPDiIvl8oEtQSnnJhxhjNgsNfUMqQ=', 'part_set_header': {'total': 1, 'hash': 'Fq1UuE5x3Co3ilPQjW6MQpyuflhlzU/mKKOx8ApcWcQ='}}, 'last_commit_hash': '/aMZnEJwASI973SgZqcTk3pMezcQOG5RaIkUQB5eFfY=', 'data_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'validators_hash': 'ZBHl+8zQcz99f5Idpo/IwysnDm4ahDjj0zpV3cXgGTk=', 'next_validators_hash': 'ZBHl+8zQcz99f5Idpo/IwysnDm4ahDjj0zpV3cXgGTk=', 'consensus_hash': '5mDvFKlRQ9sPPq8vMfF33jNN5atlBXn9CToQy66G1aY=', 'app_hash': 'jR2zzOan3Fv3WxAu2Y2OgoGOTZJ1p/kNr0YA0b+EHlo=', 'last_results_hash': 'nuSbZ/nxzFLzsX9EYS2+rMRnt9AuZtE6svfavtmRI4Y=', 'evidence_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=', 'proposer_address': 'asrigcDpNocfqmcPIglWHhehEHE='}, 'data': {'txs': []}}}





c

In [31]:
#fetch client, connection, consensus proofs



{'tx_response': {'height': '8157504', 'txhash': '3E99974EF2CBAFA472F723AB3EFDB0455296F58D7F5A349A351FEF35E07C25B9', 'codespace': '', 'code': 0, 'data': '0A250A232F6962632E636F72652E636C69656E742E76312E4D7367557064617465436C69656E74', 'raw_log': '[{"events":[{"type":"message","attributes":[{"key":"action","value":"/ibc.core.client.v1.MsgUpdateClient"},{"key":"module","value":"ibc_client"}]},{"type":"update_client","attributes":[{"key":"client_id","value":"07-tendermint-3241"},{"key":"client_type","value":"07-tendermint"},{"key":"consensus_height","value":"1-3200591"},{"key":"header","value":"0a262f6962632e6c69676874636c69656e74732e74656e6465726d696e742e76312e48656164657212af3f0af0160a90030a02080b1207706973636f2d3118cfacc301220c08e6dee99c0610b3828bf2022a480a2010d03a6ae025c6d5e18fa10f0e222f97ca04b504a79c9871863360b0d7d432a412240801122016ad54b84e71dc2a378a53d08d6e8c429cae7e5865cd4fe628a3b1f00a5c59c43220fda3199c427001223def74a066a713937a4c7b3710386e51688914401e5e15f63a20e3b0c44298fc1c149afb

In [ ]:
#fabricate message & dispatch via stargate

IBC connection step 3 - MsgConnectionOpenAck

In [ ]:
#update client on terra (ie, refresh terra's knowledge of osmosis's tendermint state) & fetch proofs

In [ ]:
#fabricate message & dispatch via stargate

IBC connection step 4 - MsgConnectionOpenConfirm

In [3]:
#update client again on osmosis & fetch proofs

In [ ]:
#fabricate message & dispatch via stargate

In [ ]:
#persist connection_id's on both chains for later stages